# Notes

## About the Dataset
The dataset set used is called Human Activity Recognition(HAR) in which the experiments have been carried out with a group of 30 volunteers within an age bracket of 19-48 years. Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING) wearing a smartphone (Samsung Galaxy S II) on the waist. Using its embedded accelerometer and gyroscope, they captured *3-axial linear acceleration* and *3-axial angular velocity* at a constant rate of 50Hz. The experiments have been video-recorded to label the data manually. The obtained dataset has been randomly partitioned into two sets, where 70% of the volunteers was selected for generating the training data and 30% the test data. 

The sensor signals *(accelerometer and gyroscope)* were pre-processed by applying noise filters and then sampled in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window). The sensor acceleration signal, which has gravitational and body motion components, was separated using a Butterworth low-pass filter into body acceleration and gravity. The gravitational force is assumed to have only low frequency components, therefore a filter with 0.3 Hz cutoff frequency was used.

## Objective
To train a model on sensor data from accelerometer and gyroscope having spacial information in 3 axis and temporal information to produce output in the form of one of the 6 classes described in the dataset.

## Implementation for cnn-lstm
There were three variables which were measured in three axis x,y,z - total acceleration, body acceleration and angular acceleration from gyroscope. These resulted in 9 files with each file having samples having 128 time steps. The data is combined from all the files each variable acting as features and converting the entire dataset into shape *no. of samples X time steps X no. features*. For training data, this shape is 7352 X 128 X 9. For each sample there is an output out of 6 activities as stated above. This ouput data is then converted into one-hot encoded data of an array of size 6.

This data is then converted into a 4D matrix for passing into the CNN architecture. **The objective of CNN architecture** is find pattern across the 3 variables and in their 3 axis & also see if there is correlation between nearby data points. To achieve this, the data is converted into the format of *no. of samples X length X width X channels*. For this dataset, the this converts into 7352 X 4 X 32 X 9. This means that for each sample, the 128 points are divided into 32 batch size and stacked one below another. 1D convolutional layers are used in this expermient because the sequence is 1D and 2D convolutional layer will not provide much meaningful information. Padding is not same for the output to avoid creating dummy data points to fill the size. For all purposes of discussion the input size taken will be for one subsequence, i.e., 1 X 1 X 32 X 9. The input is passed through 2 1D convolutional layers with kernel size 4 and no. of filters 64 and 128 respectively which produces ouput data of size 1 X 1 X 29 X 64 & 1 X 1 X 26 X 128 respectively. It is then passed through dropout layer to keep units with having 0.6 or more probability. This layer acts as regularization in the architecture to avoid overfitting. This is passed through max pool layer to reduce the no. of data points and consolidating data with output size 1 X 1 X 13 X 128 and finally through flatten layer to convert the 3 dimensions - length, width and channels into a single matrix of size 1 X 1 X 1664. This is the end of CNN pipeline.

This is passed through time distributed layer to make each flattened layer as an individual time step data otherwise the data would have been converted into the form 1 X 1664. After this it is passed through LSTM layer. **LSTM layer's objective** is to get temporal information so the data will be passed in sample each having some time steps having 1664 features. This layer has 100 units with ouput size 1 X 100 where each layer is producing hidden cell information for LSTM. This data is passed through dropout layer with probability 0.6. It is then passed through dense layer having 125 units and finally through output layer having 6 layers to produce ouput for one of the 6 classes.

## Model Used
The data was tested on two models - one having purely LSTM layer and another having CNN and LSTM layers. Both were trained on same no. of epochs, i.e., 15 and batch size 64. It was found that accuracy for LSTM model was found to be 88.8%(avg) and for CNN-LSTM model it was 90.6%(avg).

In [54]:
# For running in google colab
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Importing Libraries

In [0]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm 
from zipfile import ZipFile
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

# Downloading The Dataset

In [56]:
""" 
    check if the data (zip) file is already downloaded
    if not, download it from "https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip" 
    and save as UCI HAR Dataset.zip
"""


folder_path = 'UCI HAR Dataset'

class DownloadProgress(tqdm):
"""
    A class used to create tqdm object whichc is used to pass to the urlretrieve method to show download meter
    
    Atributes:
    ----------
    last_block: int
        no. of blocks transferred last
    block_num: int
        no. of block transferred
    block_size: float
        size of bloack in bytes
    total_size: float
        stores total size of the file
"""
    
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num


if not isfile('cifar-10-python.tar.gz'):
    with DownloadProgress(unit='B', unit_scale=True, miniters=1, desc='UCI HAR Dataset') as pbar:
        urlretrieve(
            'https://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI%20HAR%20Dataset.zip',
            'UCI HAR Dataset.zip',
            pbar.hook)

if not isdir(folder_path):
    with ZipFile('UCI HAR Dataset.zip') as zip:
        zip.extractall()
        zip.close()

UCI HAR Dataset: 61.0MB [00:01, 45.1MB/s]                           


# Loads the Data

In [0]:
"""
    Loads the data from the filepath as dataframe and returns an array of values
"""
def load_file(filepath):
    # load a single file as a numpy array
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

In [0]:
"""
    Loads groups of data from the filepath into an array and returns reshaped version of array
    
    Parameters:
    ----------
    filenames: list
        list of tring containing the file names
    prefix: str
        stores the path upto the file name
        
    Returns:
    --------
    loaded: list
        Array of transformed data
"""
def load_group(filenames, prefix=''):
    # load a list of files and return as a 3d numpy array
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

In [0]:
"""
    Loads the train and test dataset
    
    Parameters:
    ----------
    group: str
        string containing 'train' or 'test'
    prefix: str
        stores the path upto the file name
        
    Returns:
    --------
    X,y: list
        Array of input and output data
"""
def load_dataset_group(group, prefix=''):
    # load a dataset group, such as train or test
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

In [0]:
"""
   Loads the train and test input and ouput data and convert the ouput into one-hot encoded form
    
    Parameters:
    ----------
    prefix: str
        stores the path upto the file name
        
    Returns:
    --------
    trainX: list
        Array of training input
    trainy: list
        Array of training output
    testX: list
        Array of testing input
    testy: list
        Array of testing output
"""
def load_dataset(prefix=''):
    # load the dataset, returns train and test X and y elements
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

# Building The Model

In [0]:
"""
    Build the LSTM neural network, fit on training data and test the accuracy of model.
    The neural network architecture has four layers: LSTM layer, Dropout Layer, Dense layer, Ouput Layer
    For evaluation -
    No of epochs: 15
    Batch size: 64
    Loss function: categorical cross entropy(i.e. combination of softmax and cross entropy function)
    Optimizer used: Adam
    Metric for accuracy: accuracy against truth value
    Architecture -
    LSTM layer:
        no. of units: 100
        Input shape: no. of samples in each sliding frame X no. of features
    Dropout layer:
        probability for keeping the unit: 0.5
    Dense layer:
        no. of units: 100
        activation function: linear rectifier
    Output layer:
        no. of units: 6
        activation function: softmax
"""
def evaluate_model_lstm(trainX, trainy, testX, testy):
    # fit and evaluate a model
    #lstm model
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [0]:
# fit and evaluate a model
#cnn lstm model
"""
    Build the CNN LSTM neural network, fit on training data and test the accuracy of model.
    The neural network architecture has 10 layers: 1D Convolution layer, 1D Convolution layer,
    Dropout layer, 1D Maxpool layer, Time Distributed Layer, Flatten layer, LSTM layer,
    Dropout Layer, Dense layer, Ouput Layer
    For evaluation -
    No of epochs: 25
    Batch size: 64
    Loss function: categorical cross entropy(i.e. combination of softmaxx and cross entropy function)
    Optimizer used: Adam
    Metric for accuracy: accuracy against truth value
    Architecture -
    1D Convolution layer:
        converted sliding frames X sample X features into no. of inputs X length X width X features
        no. of filters: 64
        kernel size: 4
        stride: 1
        activation function: linear rectifier
        input shape: length X width X no. of features
    1D Convolution layer:
        no. of filters: 128
        kernel size: 4
        stride: 1
        activation function: linear rectifier
    Dropout layer:
        probability for keeping the unit: 0.6
    1D Maxpool layer:
        kernel size: 2
    Flatten layer:
        Flatten the 4D ouput from convolutional layers
    Time Distributed Layer:
        a wrapper around the CNN layers to make a temporal slice of the outputs from CNN and not convert them into a
        single input data
    LSTM layer:
        no. of units: 100
        activation function: tanh
    Dropout layer:
        probability for keeping the unit: 0.6
    Dense layer:
        no. of units: 125
        activation function: linear rectifier
    Output layer:
        no. of units: 6
        activation function: softmax
"""
def evaluate_model_cnn_lstm(trainX, trainy, testX, testy):
	# define model
	verbose, epochs, batch_size = 0, 25, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	# reshape data into time steps of sub-sequences
	n_steps, n_length = 4, 32
	trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
	testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
	# define model
	model = Sequential()
	model.add(TimeDistributed(Conv1D(filters=64, kernel_size=4, activation='relu'), input_shape=(None,n_length,n_features)))
	model.add(TimeDistributed(Conv1D(filters=128, kernel_size=4, activation='relu')))
	model.add(TimeDistributed(Dropout(0.6)))
	model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
	model.add(TimeDistributed(Flatten()))
	model.add(LSTM(100))
	model.add(Dropout(0.6))
	model.add(Dense(125, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

In [0]:
"""
    Prints the mean and standard deviation of the scores calculated
    
    Parameters:
    scores: list
        array of scores
    
    Return:
    -------
    None
"""
def summarize_results(scores):
    # summarize scores
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [0]:
"""
    Runs the LSTM experiment with compiling, evaluating and displaying the results
    
    Parameters:
    -----------
    repeats: int
        no. of time the model needs to be evaluated
    
    Return:
    -------
    None
"""
def run_experiment_lstm(repeats=5):
    # run a lstm experiment
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model_lstm(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [0]:
"""
    Runs the CNN-LSTM experiment with compiling, evaluating and displaying the results
    
    Parameters:
    -----------
    repeats: int
        no. of time the model needs to be evaluated
    
    Return:
    -------
    None
"""
def run_experiment_cnn_lstm(repeats=5):
    # run a cnn-lstm experiment
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model_cnn_lstm(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [70]:
# run the lstm experiment
run_experiment_lstm()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 89.379
>#2: 89.888
>#3: 88.768
>#4: 88.904
>#5: 91.381
[89.37902952154734, 89.88802171700034, 88.76823888700373, 88.90397013912454, 91.38106549032915]
Accuracy: 89.664% (+/-0.944)


In [71]:
#run the cnn lstm experiment
run_experiment_cnn_lstm()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 91.754
>#2: 90.668
>#3: 88.191
>#4: 91.144
>#5: 90.363
[91.75432643366135, 90.66847641669494, 88.19138106549033, 91.14353579911774, 90.36308109942314]
Accuracy: 90.424% (+/-1.211)
